In [1]:
import pandas as pd
import numpy as np

In [5]:
dados = pd.read_csv("dados_pre_processados.csv")
dados.drop("Unnamed: 0", axis=1, inplace=True)
dados.head()

,CodigoNotaFiscal,CodigoCentroSaida,CodigoReceita,Volume,RelacaoAguaCimento Emitida,TipoLancamento,DataEmissaoNotaFiscal,CodigoCliente,CodigoMotorista,Status,...,QuantidadeAdicaoAgua,TempoPermanenciaObra,ConsumoTotal,Idade,TipoTraco,CodigoAglomerante,FCK,RelacaoAguaCimento Receita,Rompimento7Dias,Rompimento28Dias
0,22,51020,11051,7.0,4.065,C,2010-07-02 06:29:25.063,11671051,173051,N,...,NaN,2010-07-02 01:43:00,2478.0,28,N,3051.0,15.0,0.9,10.73,21.46
1,501,51120,11051,8.0,1.011,C,2010-07-06 10:43:21.2,11575051,517051,N,...,NaN,2010-07-06,2832.0,28,N,3051.0,15.0,0.9,10.69,18.45
2,1204,51120,11051,8.0,1.011,C,2010-07-09 08:53:39.83,11003051,28079,N,...,NaN,2010-07-09 01:00:00,2832.0,28,N,3051.0,15.0,0.9,11.65,19.90
3,1713,51020,11051,6.0,1.011,C,2010-07-12 11:16:20.723,7166051,340051,N,...,NaN,2010-07-12 00:20:00,2124.0,28,N,3051.0,15.0,0.9,10.50,18.37
4,2090,51020,11051,8.0,1.011,C,2010-07-14 08:01:53.28,12403051,395051,N,...,NaN,NaN,2832.0,28,N,3051.0,15.0,0.9,11.69,19.55


In [41]:
mask_codigo_aglomerante_existente = dados["CodigoAglomerante"].notna()

dados = dados[mask_codigo_aglomerante_existente]

In [42]:
curvas_laboratorio = pd.read_csv("HistoricoCurvaAbramsEDesvios.csv",  index_col = "CodigoAglomerante")
curvas_laboratorio.head()

,Idade,AbramsX,AbramsY,FCKFaixa1,DesvioFaixa1,DesvioTEFaixa1,FCKFaixa2,DesvioFaixa2,DesvioTEFaixa2,FCKFaixa3,DesvioFaixa3,DesvioTEFaixa3,FCKFaixa4,DesvioFaixa4,DesvioTEFaixa4,DesvioMaximo,DesvioTEMaximo,DataAlteracao
CodigoAglomerante,,,,,,,,,,,,,,,,,,
13051,0,1.4763,103.500,18,5.0,NaN,25.0,4.5,NaN,34.0,4.2,NaN,NaN,NaN,NaN,5.0,NaN,2010-06-30 16:55:51.217
13051,28,-1.4763,103.500,18,5.0,NaN,25.0,4.5,NaN,34.0,4.2,NaN,NaN,NaN,NaN,5.0,NaN,2010-06-30 16:55:51.233
14051,0,-1.3574,81.545,20,3.0,NaN,25.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,2010-06-30 17:05:06.983
14051,28,-1.3574,81.545,20,3.0,NaN,25.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,2010-06-30 17:05:07
3051,0,-1.4763,103.500,18,5.0,NaN,25.0,4.5,NaN,34.0,4.2,NaN,NaN,NaN,NaN,5.0,NaN,2010-06-30 18:47:15.397


In [43]:
def EncontrarResultadoEsperadoNotaFiscal(nota):
    curvas_do_aglomerante = curvas_laboratorio.loc[nota["CodigoAglomerante"]]
    
    mask_idade_28 = curvas_do_aglomerante["Idade"] == 28
    curvas_28 = curvas_do_aglomerante[mask_idade_28]
    
    mask_data_emissao_nota = curvas_28["DataAlteracao"] <= nota["DataEmissaoNotaFiscal"]
    curvas_do_aglomerante_antes_emissao_nota = curvas_28[mask_data_emissao_nota]
    curvas_do_aglomerante_antes_emissao_nota = curvas_do_aglomerante_antes_emissao_nota.sort_values("DataAlteracao")
    
    if (curvas_do_aglomerante_antes_emissao_nota.empty):
        return np.NaN;
    
    curva_laboratorio_momento = curvas_do_aglomerante_antes_emissao_nota.iloc[len(curvas_do_aglomerante_antes_emissao_nota) - 1]    
    resultado_esperado = curva_laboratorio_momento["AbramsY"] * np.exp(curva_laboratorio_momento["AbramsX"] * nota["RelacaoAguaCimento Emitida"])
    
    return resultado_esperado;

In [44]:
dados["ResultadoEsperado"] = dados.apply(EncontrarResultadoEsperadoNotaFiscal, axis = "columns")

<ipython-input-44-66f2bf60c27b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados["ResultadoEsperado"] = dados.apply(EncontrarResultadoEsperadoNotaFiscal, axis = "columns")


In [46]:
dados.tail()

,CodigoNotaFiscal,CodigoCentroSaida,CodigoReceita,Volume,RelacaoAguaCimento Emitida,TipoLancamento,DataEmissaoNotaFiscal,CodigoCliente,CodigoMotorista,Status,...,TempoPermanenciaObra,ConsumoTotal,Idade,TipoTraco,CodigoAglomerante,FCK,RelacaoAguaCimento Receita,Rompimento7Dias,Rompimento28Dias,ResultadoEsperado
142490,556368,79020,40241051,6.5,0.647,B,2019-12-27 10:46:46.207,101868998,113079,N,...,2019-12-27 00:51:00,3810.0,28,E,85051.0,25.0,0.600,21.96,41.41,35.715339
142491,556383,79020,40241051,6.5,0.647,B,2019-12-27 12:15:46.467,101868998,160079,N,...,2019-12-27 00:25:00,3810.0,28,E,85051.0,25.0,0.600,30.13,39.51,35.715339
142492,556440,51020,47201051,8.0,0.858,B,2019-12-30 07:56:30.76,855764998,992051,N,...,2019-12-30 00:40:00,3792.0,28,E,106051.0,25.0,0.599,20.34,32.30,32.383433
142493,556450,51020,47201051,8.0,0.858,B,2019-12-30 08:31:27.947,855764998,643051,N,...,2019-12-30 00:55:00,3792.0,28,E,106051.0,25.0,0.599,20.94,33.66,32.383433
142494,556473,51020,47201051,4.0,0.858,B,2019-12-30 10:37:31.567,855764998,873051,N,...,2019-12-30 01:04:00,1896.0,28,E,106051.0,25.0,0.599,20.29,33.23,32.383433


In [47]:
dados.to_csv("dados_com_resultado_esperado.csv")